In [1]:
from datasets import load_dataset, Dataset
import gc
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
from peft import LoraConfig, AutoPeftModelForCausalLM
from trl import SFTConfig, SFTTrainer

In [2]:
dataset = load_dataset('iamjoon/klue-mrc-ko-rag-dataset', split='train')
print(set(dataset['type']))
dataset = dataset.class_encode_column('type')

{'mrc_question', 'paraphrased_question', 'no_answer', 'mrc_question_with_1_to_4_negative', 'synthetic_question'}


In [3]:
# cast_column 메서드 활용도 가능
type_name_dict = {1:'mrc_question_with_1_to_4_negative', 3:'paraphrased_question', 2:'no_answer', 4:'synthetic_question', 0:'mrc_question'}

In [4]:
system_message = '''
당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.

다음의 지시사항을 따르십시오.
1. 질문과 검색 결과를 바탕으로 답변하십시오.
2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.
3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문에 대한 내용이 없습니다." 라고 답변하십시오.
4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.
5. 예를 들어 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]라고 기재하십시오.
6. 최대한 다수의 문서를 인용하여 답변하십시오.

검색 결과:
----------
{search_result}'''

In [5]:
print('원본 데이터의 type 분포:')
for type_name in set(dataset['type']):
    print(f'{type_name_dict[type_name]}: {dataset['type'].count(type_name)}')

원본 데이터의 type 분포:
mrc_question: 491
mrc_question_with_1_to_4_negative: 296
no_answer: 404
paraphrased_question: 196
synthetic_question: 497


In [6]:
split_dataset = dataset.train_test_split(test_size=0.5, stratify_by_column='type')
train_dataset_format = split_dataset['train']
test_dataset_format = split_dataset['test']

In [7]:
def format_data(sample):
    search_result = '\n------\n'.join(f'문서{idx+1}: {result}' for idx, result in enumerate(sample['search_result']))

    return {
        'messages':[{'role':'system', 'content':system_message.format(search_result=search_result)},
                    {'role':'user', 'content':sample['question']},
                    {'role':'assistant', 'content':sample['answer']}]
    }

In [8]:
train_dataset = [format_data(train_data) for train_data in train_dataset_format]
test_dataset = [format_data(test_data) for test_data in test_dataset_format]

In [9]:
print(f'\n전체 데이터 분할 결과: Train {len(train_dataset)}개, Test {len(test_dataset)}개')
print('--'*20)
print('\n학습 데이터의 type 분포:')
for type_name in set(dataset['type']):
    print(f'{type_name_dict[type_name]}: {train_dataset_format['type'].count(type_name)}')
print('\n테스트 데이터의 type 분포:')
for type_name in set(dataset['type']):
    print(f'{type_name_dict[type_name]}: {test_dataset_format['type'].count(type_name)}')


전체 데이터 분할 결과: Train 942개, Test 942개
----------------------------------------

학습 데이터의 type 분포:
mrc_question: 245
mrc_question_with_1_to_4_negative: 148
no_answer: 202
paraphrased_question: 98
synthetic_question: 249

테스트 데이터의 type 분포:
mrc_question: 246
mrc_question_with_1_to_4_negative: 148
no_answer: 202
paraphrased_question: 98
synthetic_question: 248


In [10]:
train_dataset[345]['messages']

[{'role': 'system',
  'content': '\n당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.\n\n다음의 지시사항을 따르십시오.\n1. 질문과 검색 결과를 바탕으로 답변하십시오.\n2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.\n3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문에 대한 내용이 없습니다." 라고 답변하십시오.\n4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.\n5. 예를 들어 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]라고 기재하십시오.\n6. 최대한 다수의 문서를 인용하여 답변하십시오.\n\n검색 결과:\n----------\n문서1: 경남지역에서 운수업에 종사하는 A씨는 작년 상반기 교통사고를 당했다. 문제는 상대방이 자동차보험에 가입하지 않았다는 것이었다. 그는 사고를 낸 상대방을 보험에 가입한 지인으로 바꿔치기 했다. A씨는 병원에서 후유장애 판정을 받아 총 3억원의 보험금을 수령했다. 묻힐 뻔했던 이 보험사기는 A씨 주변인의 신고로 들통났다. 보험사는 A씨 사건을 경찰에 통보하는 한편 신고자에게 3073만원을 포상금으로 지급했다. 주부인 B씨는 고혈압을 앓고 있다는 사실을 속이고 2000년 보장성 보험에 가입했다. 2002년 가벼운 뇌경색으로 진단받아 수년간 입·통원 치료를 반복해 총 1억원의 보험금을 타냈다. 이 과정에서 환자 관리가 허술한 병원만 골라 다녔다. 제보자는 B씨의 혈색이 좋은데도 장기 입원하는 점이 의심스러워 보험범죄 신고센터에 알렸다. 보험사는 B씨가 병력을 속이고 보험에 가입했다는 이유로 경찰에 알렸고 제보자에게는 100만원의 포상금을 줬다. 보험사기가 많아지면서 제보자에게 지급하는 포상금도 급증하고 있다. 금융감독원은 지난해 보험범

In [11]:
train_dataset = Dataset.from_list(train_dataset)
test_dataset = Dataset.from_list(test_dataset)

In [12]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [13]:
model_id = 'Qwen/Qwen2.5-3B-Instruct'

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map='auto'
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
peft_config = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.1,
    r=8,
    bias='none',
    target_modules=['q_proj', 'v_proj'],
    task_type='CAUSAL_LM'
)

In [15]:
args = SFTConfig(
    output_dir='qwen_2.5-3b_rag-ko',
    num_train_epochs=2,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    optim='adamw_torch_fused',
    logging_steps=10,
    save_strategy='steps',
    save_steps=100,
    bf16=True,
    fp16=False,
    learning_rate=1e-4,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type='constant',
    push_to_hub=False,
    remove_unused_columns=False,
    report_to=None,
    completion_only_loss=True
)

In [16]:
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    peft_config=peft_config
)

Tokenizing train dataset:   0%|          | 0/942 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/942 [00:00<?, ? examples/s]

In [17]:
trainer.train()
trainer.save_model()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.
c:\Coding\Local\exercise_book_ver2\.venv\Lib\site-packages\torch\_dynamo\eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,2.257100
20,1.991900
30,1.837100
40,1.770300
50,1.597700
60,1.591800
70,1.564500
80,1.615900
90,1.566700
100,1.601900


c:\Coding\Local\exercise_book_ver2\.venv\Lib\site-packages\torch\_dynamo\eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
c:\Coding\Local\exercise_book_ver2\.venv\Lib\site-packages\torch\_dynamo\eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return 

In [18]:
del model
del trainer
gc.collect()
torch.cuda.empty_cache()

In [19]:
prompt_list = []
label_list = []

for prompt in test_dataset['messages']:
    text = tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=False)
    
    input = text.split('<|im_start|>assistant')[0] + '<|im_start|>assistant'
    label = text.split('<|im_start|>assistant')[1]

    prompt_list.append(input)
    label_list.append(label)


In [20]:
print(prompt_list[34])
print('--'*30)
print('--'*30)
print('--'*30)

<|im_start|>system

당신은 검색 결과를 바탕으로 질문에 답변해야 합니다.

다음의 지시사항을 따르십시오.
1. 질문과 검색 결과를 바탕으로 답변하십시오.
2. 검색 결과에 없는 내용을 답변하려고 하지 마십시오.
3. 질문에 대한 답이 검색 결과에 없다면 검색 결과에는 "해당 질문에 대한 내용이 없습니다." 라고 답변하십시오.
4. 답변할 때 특정 문서를 참고하여 문장 또는 문단을 작성했다면 뒤에 출처는 이중 리스트로 해당 문서 번호를 남기십시오. 예를 들어 특정 문장이나 문단을 1번 문서에서 인용했다면 뒤에 [[ref1]]이라고 기재하십시오.
5. 예를 들어 특정 문장이나 문단을 1번 문서와 5번 문서에서 동시에 인용했다면 뒤에 [[ref1]], [[ref5]]라고 기재하십시오.
6. 최대한 다수의 문서를 인용하여 답변하십시오.

검색 결과:
----------
문서1: 2009년 8월 관객 수 1000만명을 넘어 흥행 질주하던 영화 ‘해운대’가 갑자기 중국 온라인에서 불법복제돼 나돌기 시작했다. 투자배급사인 CJ E&M은 이를 차단하기 위해 파트너사인 베이징문전세기문화전매유한공사 측에 알렸다. 유한공사 측은 단속에 나서기 전에 영화의 저작권자가 누구인지 한국저작권위원회에 문의했다. 위원회가 저작권자는 CJ라는 인증서를 전달하자 유한공사는 이를 근거로 불법복제물을 단속했다. 한국저작권위원회는 국내외에서 콘텐츠 거래를 활성화하기 위해 저작권인증서비스를 시행하고 있다. 공신력 있는 인증기관이 저작물의 권리자가 누구인지 확인해주는 서비스다. 외국인 입장에서는 한국 영화와 드라마 등을 구입하려고 해도 저작권자가 누구인지 몰라 사지 못하는 사례가 많다. 저작권자를 사칭한 사기꾼들에게 피해를 보는 경우도 많다. 중국에서는 한국저작권위원회가 국가판권국(한국의 문화체육관광부에 해당)의 정식 허가를 받고 중국 내에서 유통되는 한국 저작물의 권리관계를 확인해주고 있다. 국내 저작물을 중국에 수출하려면 중국 판권국 산하기관인 중국판권보호중심으로부터 등록번호를 취득

In [21]:
print(label_list[34])


검색 결과에는 청소년활동진흥법 초안 작성을 담당한 국회의원을 찾을 수 없습니다.<|im_end|>



In [22]:
del tokenizer
gc.collect()
torch.cuda.empty_cache()

In [23]:
base_model_id = 'Qwen/Qwen2.5-3B-Instruct'
base_model = AutoModelForCausalLM.from_pretrained(base_model_id, device_map='auto', dtype=torch.bfloat16, quantization_config=bnb_config)
base_tokenizer = AutoTokenizer.from_pretrained(base_model_id)
pipe = pipeline('text-generation', model=base_model, tokenizer=base_tokenizer)
eos_token = base_tokenizer('<|im_end|>', add_special_tokens=False)['input_ids'][0]

def test_inference(pipe, prompt):
    outputs = pipe(prompt, max_new_tokens=1024, eos_token_id=eos_token, do_sample=False)
    
    return outputs[0]['generated_text'][len(prompt):].strip()

prompt = prompt_list[34]
label = label_list[34]
pred = test_inference(pipe, prompt)

print(f'모델의 예측:\n{pred}')
print(f'정답:\n{label}')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


모델의 예측:
검색 결과에서는 해당 정보가 명시되어 있지 않습니다. 이 문제에 대한 답변을 드릴 수 없습니다. [[ref1]], [[ref2]], [[ref3]], [[ref4]], [[ref5]]에서 해당 정보를 찾을 수 없습니다.
정답:

검색 결과에는 청소년활동진흥법 초안 작성을 담당한 국회의원을 찾을 수 없습니다.<|im_end|>



In [24]:
del base_model
gc.collect()
torch.cuda.empty_cache()

In [25]:
peft_model_id = 'qwen_2.5-3b_rag-ko/checkpoint-236'
peft_model = AutoPeftModelForCausalLM.from_pretrained(peft_model_id, device_map='auto', dtype=torch.bfloat16, quantization_config=bnb_config)
pipe = pipeline('text-generation', model=peft_model, tokenizer=base_tokenizer)

prompt = prompt_list[34]
label = label_list[34]
pred = test_inference(pipe, prompt)

print(f'모델의 예측:\n{pred}')
print(f'정답:\n{label}')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


모델의 예측:
현재로서는 해당 정보를 찾을 수 없습니다 ([[ref1]])
정답:

검색 결과에는 청소년활동진흥법 초안 작성을 담당한 국회의원을 찾을 수 없습니다.<|im_end|>



In [26]:
del peft_model
del base_tokenizer
gc.collect()
torch.cuda.empty_cache()

In [27]:
def check_gpu_memory_leaks():
    print("=== GPU 메모리 점유 중인 텐서 목록 ===")
    count = 0
    total_mem = 0
    
    # 가비지 컬렉터가 관리하는 모든 객체를 스캔
    for obj in gc.get_objects():
        try:
            # 텐서 객체인지 확인 (파라미터 포함)
            if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
                
                # GPU(cuda)에 있는지 확인
                if obj.is_cuda:
                    # 텐서 크기 계산
                    mem = obj.element_size() * obj.nelement()
                    total_mem += mem
                    
                    # 너무 작은 건 생략하고 큰 것만 출력 (예: 1MB 이상)
                    if mem > 1024 * 1024: 
                        print(f"Shape: {obj.shape}, Size: {mem/1024**2:.2f} MB, Type: {obj.dtype}")
                        # 어떤 변수가 잡고 있는지는 알기 어렵지만, shape를 보면 뭔지 짐작 가능
                        # 예: (32000, 4096) -> 이건 모델의 임베딩 레이어구나!
                    
                    count += 1
        except:
            pass # 접근 불가능한 객체는 패스
            
    print(f"--- 총 발견된 텐서 개수: {count}")
    print(f"--- 총 추정 메모리 사용량: {total_mem / 1024**3:.2f} GB")

# 실행
check_gpu_memory_leaks()

=== GPU 메모리 점유 중인 텐서 목록 ===
Shape: torch.Size([151936, 2048]), Size: 593.50 MB, Type: torch.bfloat16
Shape: torch.Size([2097152, 1]), Size: 2.00 MB, Type: torch.uint8
Shape: torch.Size([11272192, 1]), Size: 10.75 MB, Type: torch.uint8
Shape: torch.Size([352256]), Size: 1.34 MB, Type: torch.float32
Shape: torch.Size([11272192, 1]), Size: 10.75 MB, Type: torch.uint8
Shape: torch.Size([352256]), Size: 1.34 MB, Type: torch.float32
Shape: torch.Size([11272192, 1]), Size: 10.75 MB, Type: torch.uint8
Shape: torch.Size([352256]), Size: 1.34 MB, Type: torch.float32
Shape: torch.Size([2097152, 1]), Size: 2.00 MB, Type: torch.uint8
Shape: torch.Size([11272192, 1]), Size: 10.75 MB, Type: torch.uint8
Shape: torch.Size([352256]), Size: 1.34 MB, Type: torch.float32
Shape: torch.Size([11272192, 1]), Size: 10.75 MB, Type: torch.uint8
Shape: torch.Size([352256]), Size: 1.34 MB, Type: torch.float32
Shape: torch.Size([11272192, 1]), Size: 10.75 MB, Type: torch.uint8
Shape: torch.Size([352256]), Size: 1.34

c:\Coding\Local\exercise_book_ver2\.venv\Lib\site-packages\torch\__init__.py:1136: FutureWarning: `torch.distributed.reduce_op` is deprecated, please use `torch.distributed.ReduceOp` instead
  return isinstance(obj, torch.Tensor)
C:\Users\Admin\AppData\Local\Temp\ipykernel_2420\2414447976.py:10: FutureWarning: `torch.distributed.reduce_op` is deprecated, please use `torch.distributed.ReduceOp` instead
  if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
c:\Coding\Local\exercise_book_ver2\.venv\Lib\site-packages\torch\_inductor\ops_handler.py:745: UserWarning: undefined OpHandler.data, please add missing op schema
  warnings.warn(f"undefined OpHandler.{name}, please add missing op schema")
